In [96]:
import pandas as pd
import numpy as np

Load the movies and ratings data into Pandas dataframes:

In [97]:
movies = pd.read_csv('movies2.csv')  # Movie information, including genres
ratings = pd.read_csv('ratings2.csv')  # Ratings by users

The pivot table will have movie titles as the index, userId as columns, and values as the ratings given by users. Use NaN for missing ratings (where the user hasn’t rated a movie).

In [98]:
# Merge movies and ratings
movie_ratings = pd.merge(movies, ratings, on='movieId')

# Create the pivot table (movie_matrix)
movie_matrix = movie_ratings.pivot_table(index='title', columns='userId', values='rating')

In [99]:
target_movie = "Titanic (1997)"

Extract the user ratings for the movie you want to base your recommendations on:

In [100]:
target_movie_ratings = movie_matrix.loc[target_movie]

In [101]:
print(target_movie in movie_matrix.index)

True


In [102]:
# Check if the target movie exists in the matrix
if target_movie in movie_matrix.index:
    # Get ratings for the target movie
    target_movie_ratings = movie_matrix.loc[target_movie]

    # Calculate correlations with other movies
    similar_movies = movie_matrix.corrwith(target_movie_ratings)

    # Drop NaN values and convert to DataFrame
    similar_movies = similar_movies.dropna().to_frame(name='correlation')

    # Calculate movie stats and merge
    movie_stats = movie_ratings.groupby('title').agg({'rating': ['mean', 'count']})
    movie_stats.columns = ['mean_rating', 'num_ratings']
    similar_movies = similar_movies.join(movie_stats['num_ratings'])

    # Check if there are any movies with a sufficient number of ratings
    if not similar_movies.empty:
        # Filter movies by number of ratings
        similar_movies = similar_movies[similar_movies['num_ratings'] > 10]

        # Sort by correlation
        similar_movies = similar_movies.sort_values('correlation', ascending=False)
        print(similar_movies)
    else:
        print("No similar movies found with sufficient ratings.")
else:
    print(f"The movie '{target_movie}' does not exist in the dataset or has insufficient ratings.")



No similar movies found with sufficient ratings.
